<center><img src="image/logo.png" alt="logo.png" border="0" width="500"/></center>
<center><b>mdapy</b>: Molecular dynamics simulation analysis with Python</center>
<center> <b>Author</b>：Yong-Chao Wu </center>
<center> <b>Supervisor</b>: Jian-Li Shao</center>
<center> <b>Date</b>: 2023/11/24</center>

# 引用

>```bibtex
@article{mdapy2023,
title = {mdapy: A flexible and efficient analysis software for molecular dynamics simulations}, 
journal = {Computer Physics Communications},
pages = {108764},
year = {2023},
issn = {0010-4655},
doi = {https://doi.org/10.1016/j.cpc.2023.108764},
url = {https://www.sciencedirect.com/science/article/pii/S0010465523001091},
author = {Yong-Chao Wu and Jian-Li Shao},
keywords = {Simulation analysis, Molecular dynamics, Polycrystal, TaiChi, Parallel computing}
}
```

# 相关链接

- Homepage: [https://github.com/mushroomfire/mdapy](https://github.com/mushroomfire/mdapy)
- Documentation: [https://mdapy.readthedocs.io](https://mdapy.readthedocs.io/)

# 什么是mdapy

**mdapy是一个高性能的分子动力学前/后处理的Python库。**

# 优点 V.S. 缺点

> **优点**

- 跨平台，安装简单，语法简单
- 支持多核CPU并行和GPU计算
- 数据存储格式为Numpy的NDARRY和Polars的DataFrame, 可以无缝衔接Python科学计算生态
- 具有非常快的文件I/O性能，良好的计算性能
- 支持正交/斜盒子体系
- 支持周期性边界

> **缺点**

- 主要面向固体材料的模拟，特别是金属/合金体系, 但设计架构上可扩展到任何全原子体系
- 适合于单一Frame的系统状态分析，不擅长一个轨迹中不同Frame间的关联分析
- 并行方式为共享内存并行，目前不支持跨节点并行
- 可视化功能还处在实验性阶段
- 支持的文件格式较少，目前直接支持LAMMPS中的DUMP和DATA格式，VASP中的POSCAR格式，通用的XYZ格式，其他格式需要手动转换

<img src="image/bench.jpg" alt="bench.jpg" border="0" width="800"/>

# 安装

> ```bash
> conda create -n mda python=3.10
> conda activate mda
> pip install mdapy -U 
> pip install mdapy[all] -U # If you want to install all optional dependencies```

# 主要功能

<img src="image/main_feature.jpg" alt="main_feature.jpg" border="0" width="800"/>

# 工作流示意图

<img src="image/workflow.png" alt="workflow.png" border="0" width="800"/>

# 新手教程

默认会导入如下库并采用常用的简称

In [ ]:
import mdapy as mp
import numpy as np
import polars as pl
from tqdm import tqdm
import matplotlib.pyplot as plt
mp.init()
print(f'Version of mdapy is: {mp.__version__}')

## System 类

mdapy中一个核心的模块是**System**类，这个类由原子的Position, Box和Boundary组成。
生成System类有许多方式。

第一种方式，直接读取一个Dump/Data文件：

In [ ]:
system = mp.System('./frame/CoCuFeNiPd.dump')

### 查看system实例包含的信息

In [ ]:
system

### 查看Box信息

1. mdapy中的盒子是一个(4x3)的2维数组，前三行代表盒子的三个基向量，最后一行是盒子的原点
2. 盒子采用[LAMMPS的规则](https://docs.lammps.org/Howto_triclinic.html)，所以box[0, 1]==box[0, 2]==box[1, 2]==0
3. 对于正交盒子，为了和之前版本的兼容性，mdapy也支持(3x2)的盒子，第一列代表[xlo, ylo, zlo],第二列代表[xhi, yhi, zhi]

In [ ]:
system.box

### 查看原子Position信息

1. 位置信息存储在一个(Nx3)的2维数组中, 从左到右每一列分别代表x, y, z坐标
2. 仅提供一个view, 用户不应该直接修改位置信息

In [ ]:
system.pos

### 查看边界Boundary信息

1. 边界条件由一个包含三个数字的列表表示，其中1代表周期性边界，0代表自由边界

In [ ]:
system.boundary

### 查看所有Particle相关的数据

1. 粒子携带的所有信息都存储在一个DataFrame中
2. 用户可以很方便的对每一个属性，每一个原子进行统计分析

In [ ]:
system.data.head()

## 邻域信息

当用户实例化一个System类以后，就可以进行后续的大多数分析计算。对于mdapy中的所有分析，可以大致分为两类：

1. 需要邻域信息的
2. 不需要邻域信息的

### 固定截断距离的邻域

实际上这是所有粒子模拟都会面临的一个共同问题，如何快速的找到一个粒子周围和它距离小于rc的所有粒子。
最简单的做法，直接使用双重循环，这样的复杂度是$O(N^2)$。一个常见的优化方法是使用linked-cell-list算法,
简单来说就是首先将粒子划分到三维的网格中，这样搜索邻域只需要在周围的27个网格中检索就行了，大大降低了复杂程度。

当然，这里面如何划分网格宽度？也会对计算效率带来一定的影响。以及如果粒子带有不同的半径，这时候可能直接划分网格就会效率很低了。

1. 在mdapy中，网格的长度是截断半径加一个小量，采用元胞链表算法，考虑周期性边界条件。
2. 我们对程序进行了一些优化，比如对于初始网络划分部分，是在C++端完成的，后续的并行检索是在Taichi端完成的。

与Ovito和Freud的效率对比，以及本身的并行扩展性对比如下：

<img src="image/neighbor.jpg" alt="neighbor,jpg" border="0" width="800"/>

In [ ]:
system.build_neighbor(rc=5., max_neigh=60)

- 查看原子的邻域列表

1. 是一个(N, max_neigh)的2维数组，每一行对应该原子的邻域内原子索引
2. max_neigh代表最大邻域原子数目，可以手动指定，也可以自动生成
3. 手动指定可以节约内存，提高计算速度
4. 其中-1是默认填充值，原子索引从0开始

In [ ]:
system.verlet_list

- 查看原子的邻域距离列表
  
1. 与verlet_list相对应的欧式距离，考虑了周期性边界
2. verlet_list中-1对应的地方默认填充为rc+1的数值，这样处理是为了方便排序

In [ ]:
system.distance_list

- 查看原子的邻域原子数目

1. 数字代表每个原子的实际邻域原子数，这些数都<=max_neigh

In [ ]:
system.neighbor_number

In [ ]:
assert system.neighbor_number.max() <= 60

- 如果我们想获取某一个原子的邻域列表，以0号原子为例

In [ ]:
system.verlet_list[0][system.verlet_list[0]>-1]

- 对应的距离如下

In [ ]:
system.distance_list[0][system.verlet_list[0]>-1]

- 计算距离来验证

In [ ]:
system.atom_distance(0, 8787)

### 固定数目的最近邻查找

另一种常见的情况就是我不关心离我多近的有多少，我只想要距离我最近的多少个粒子信息。

对于直接查找，最常用的是使用Tree的数据结构，常见的有kdtree, aabb tree, vptree, BVH tree等。

在mdapy中的处理方式如下：

1. 如果System没有Neighbor，对于正交盒子会创建一个kdtree来进行查找，对于斜盒子则会使用网格法迭代查找
2. 如果System有Neighbor，且最小邻域原子数目不小于待查找数目，则对邻域进行部分排序
3. 一般来说，需要进行多种分析时，先建立一个较大的邻域，可以有效的节约总计算时间

In [ ]:
kdt = mp.NearestNeighbor(system.pos, system.box, system.boundary)

In [ ]:
distance_list, verlet_list = kdt.query_nearest_neighbors(12)

- 查看最近邻信息

In [ ]:
verlet_list[0]

In [ ]:
distance_list[0]

## 后处理分析

- 当用户实例化System以后，就可以使用?来检索所有可用的分析方法。

1. 对于计算结果为粒子属性的，如温度，中心对称参数等，mdapy会把计算结果直接添加到system.data中。
2. 对于计算结果不是粒子属性的，如径向分布函数，WCP参数等，mdapy会保留对应的实例。

### 计算径向分布函数

In [ ]:
system.cal_pair_distribution()

这里就生成了PairDistribution实例，一般生成的实例名称和调用的函数名称接近，也可以使用?查看。

In [ ]:
fig, ax = system.PairDistribution.plot()

In [ ]:
fig, ax = system.PairDistribution.plot_partial()

### 计算WCP参数

In [ ]:
system.cal_warren_cowley_parameter()

In [ ]:
system.WarrenCowleyParameter.WCP

In [ ]:
fig, ax = system.WarrenCowleyParameter.plot()

### PTM结构分析

In [ ]:
system.cal_polyhedral_template_matching()

### 计算中心对称参数

In [ ]:
system.cal_centro_symmetry_parameter()

### 计算构型熵

In [ ]:
system.cal_atomic_entropy(rc=3.6*1.4, compute_average=True, average_rc=3.6*0.9)

### 计算Voronoi体积

In [ ]:
system.cal_voronoi_volume()

### 查看计算结果

In [ ]:
system.data.head()

### 可视化结果

In [ ]:
system.display()

这里默认使用structure type对原子上色，也可以使用任意其他属性，比如原子type，原子体积等

In [ ]:
system.atoms_colored_by('type')

In [ ]:
system.atoms_colored_by('voronoi_volume')

### 保存结果到本地

- 保存粒子属性可以使用dump或者xyz格式

In [ ]:
system.write_dump()

In [ ]:
system.write_xyz()

### 判断FCC晶体是solid还是liquid

- 这里使用一个固液共存的dump文件为例子，元素为钼 Molybdenum 元素，熔点约2896 K
- 其原理是基于Steinhardt Bond order

In [ ]:
system = mp.System('frame/solidliquid.dump')

In [ ]:
system.cal_steinhardt_bond_orientation(solidliquid=True)

### 计算原子平均温度

In [ ]:
system.cal_atomic_temperature(elemental_list=['Mo'])

In [ ]:
print(f"Average temperature is {system.data['atomic_temp'].mean()} K")

In [ ]:
system.data.head()

In [ ]:
system.atoms_colored_by('atomic_temp')

In [ ]:
system.atoms_colored_by('solidliquid')

In [ ]:
system.write_dump()

### 筛选原子可视化

In [ ]:
solid = system.select(system.data.filter(pl.col('solidliquid')==1))

In [ ]:
solid.display()

### 识别FCC基体中的ISF, ESF, TW, HCP

- 这里使用一个包含多种缺陷的dump文件

In [ ]:
system = mp.System('frame/ISF.dump')

In [ ]:
system.cal_identify_SFs_TBs()

In [ ]:
system.display()

In [ ]:
system.atoms_colored_by('fault_types')

In [ ]:
system.write_dump()

### 斜盒子体系

- 大多数分析方式对斜盒子体系依然适用
- 这里读取一个HCP的Ti体系

In [ ]:
Ti = mp.System('./frame/Ti.data')

In [ ]:
Ti

In [ ]:
Ti.cal_common_neighbor_analysis(rc=2.9357*1.207)

In [ ]:
Ti.cal_centro_symmetry_parameter()

In [ ]:
Ti.data

In [ ]:
Ti.display()

In [ ]:
Ti.write_dump()

### 建立初始模型

1. mdapy可以建立一些简单的模型，比如FCC, BCC, HCP, Graphene，对于FCC和BCC还支持不同晶相
2. mdapy可以建立多晶，以及带石墨烯晶界的多晶
3. 用户可以手动提供元胞信息，然后适用mdapy扩胞，再进行筛选，比如建立任意类型的纳米颗粒等

In [ ]:
fcc = mp.LatticeMaker(3.615, 'FCC', 10, 10, 10)
fcc.compute()

In [ ]:
fcc.box

In [ ]:
fcc.pos

保存为data/dump文件

In [ ]:
fcc.write_data(output_name='./frame/fcc.data')

我们来可视化一下结果，首先需要使用position和box来实例化一个System类

In [ ]:
system = mp.System(pos=fcc.pos, box=fcc.box)

In [ ]:
system

In [ ]:
system.display()

对于FCC和BCC晶体可以指定初始晶向

In [ ]:
fcc_111 = mp.LatticeMaker(3.615, 'FCC', 10, 10, 10, crystalline_orientation=np.array([[1,1,1], [1,-1,0], [1,1,-2]]))
fcc_111.compute()

In [ ]:
fcc_111.write_dump(output_name='./frame/fcc_111.dump')

In [ ]:
system = mp.System(pos=fcc_111.pos, box=fcc_111.box)

In [ ]:
system.display()

mdapy处理几百万原子体系也很轻松，这里我们简单进行扩胞，生成一个400w原子体系进行测试

In [ ]:
%%time
repli = mp.Replicate(fcc.pos, fcc.box, 10, 10, 10)
repli.compute()

In [ ]:
repli.N

建立$rc=0.5nm$的邻域

In [ ]:
%%time
neigh = mp.Neighbor(repli.pos, repli.box, 5., max_neigh=50)
neigh.compute()

手动排序获取12个最近邻原子

In [ ]:
%%time
neigh.sort_verlet_by_distance(12)

In [ ]:
neigh.verlet_list[0][:12]

In [ ]:
neigh.distance_list[0][:12]

使用kdtree来搜索12个最近邻原子

In [ ]:
%%time
kdt = mp.NearestNeighbor(repli.pos, repli.box)
distance_list, verlet_list = kdt.query_nearest_neighbors(12)

In [ ]:
verlet_list[0]

In [ ]:
distance_list[0]

### 建立纳米晶

In [ ]:
box = np.array([[200, 0, 0],
               [0, 200, 0],
               [0, 0, 200],
               [0, 0, 0.]])

In [ ]:
poly = mp.CreatePolycrystalline(box, 20, 3.615, 'FCC', output_name='./frame/poly.dump', metal_overlap_dis=2.)
poly.compute()

使用DataFrame来实例化System

In [ ]:
system = mp.System(data=poly.data, box=box)

In [ ]:
system

In [ ]:
system.atoms_colored_by('grainid')

### EAM势函数分析

#### 生成一个Al的势函数

- mdapy可以基于文献生成一元或多元的eam.alloy势函数，可用于LAMMPS计算
- 可选元素列表 ["Cu","Ag","Au","Ni","Pd","Pt","Al","Pb","Fe","Mo","Ta","W","Mg","Co","Ti","Zr"]

In [ ]:
mp.EAMGenerate(['Al'], output_name='./frame/Al.eam.alloy');

#### 读取刚才生成的势函数

In [ ]:
potential = mp.EAM('./frame/Al.eam.alloy')

#### 查看势函数的相关信息

In [ ]:
potential.plot()

#### 使用势函数计算Al的EOS曲线

In [ ]:
energy = []
lattice_constant = 4.05
fcc = mp.LatticeMaker(lattice_constant, 'FCC', 1, 1, 1)
fcc.compute()
for scale in tqdm(np.arange(0.95, 1.06, 0.01)):
    Cal = mp.Calculator(potential, fcc.pos * scale, [1, 1, 1], fcc.box * scale, ['Al'], np.ones(fcc.N, int))
    Cal.compute()
    energy.append([lattice_constant*scale, Cal.energy.mean()])
energy = np.array(energy)

In [ ]:
fig, ax = mp.set_figure(figsize=(8, 6), use_pltset=True)
plt.plot(energy[:, 0], energy[:, 1], 'o-')
plt.xlabel('a ($\mathregular{\AA}$)')
plt.ylabel('PE (eV/atom)')

### 分析化学产物

mdapy目前判断两个原子的连接性的是比较经验的，当两个原子的距离小于其范德华半径之和的0.6倍，
则认为两个原子成键，然后基于成键关系来进行聚合，得出产物的种类和数目。

- 这里读取一个AP+Al的文件作为例子
- 其中的元素顺序为["H", "C", "N", "O", "F", "Al", "Cl"]

In [ ]:
system = mp.System('frame/ap@al.dump')

In [ ]:
system.display()

我们可以首先看一下这里面包含的最多的产物是那些，这里查看了数目前20的产物

In [ ]:
species = system.cal_species_number(element_list=["H", "C", "N", "O", "F", "Al", "Cl"], check_most=20)

In [ ]:
species

我们可以看到，最多的是H2O, Cl原子，N2, HCl, CO2 大概确定以后，我们可以实际输入我们关心的产物

In [ ]:
species = system.cal_species_number(element_list=["H", "C", "N", "O", "F", "Al", "Cl"], 
                                    search_species=['H2O', 'Cl', 'N2', 'CO2', 'HCl', 'X'])

这里'X'元素是一个虚构的元素，也就是如果你想要的产物没有，则会返回数目为0

In [ ]:
species

### 区域划分统计

mdapy可以对体系进行1维，2维，3维网格划分，并对里面的物理量进行平均，求和，最大值，最小值计算等操作

- 这里我们以温度为例

首先计算原子的平均温度

In [ ]:
system.cal_atomic_temperature(elemental_list=["H", "C", "N", "O", "F", "Al", "Cl"])

In [ ]:
print(f"The mean temperature is {system.data['atomic_temp'].mean()} K.")

In [ ]:
system.atoms_colored_by('atomic_temp', vmin=1000, vmax=3000)

对温度在xy平面进行二维划分并求平均

In [ ]:
system.spatial_binning('xy', 'atomic_temp')

In [ ]:
T = system.Binning.res[:, :, -1]
x, y = system.Binning.coor['x'], system.Binning.coor['y']
X, Y = np.meshgrid(x, y, indexing='ij')

In [ ]:
fig, ax = mp.set_figure(figsize=(8, 7.5), use_pltset=True)
h = ax.contourf(X, Y, T, cmap='hot', levels=100)
bar = plt.colorbar(h, ax=ax)
bar.set_label('T (K)')
ax.set_xlabel('x ($\mathregular{\AA}$)')
ax.set_ylabel('y ($\mathregular{\AA}$)')

### 处理轨迹

如前文所说，mdapy并不是为了处理轨迹类文件的，mdapy提供的分析功能中只有计算MSD和Lindemann系数需要考虑不同frame间的关系。
对于计算RDF这种，实际上可以通过一个简单的for-loop来进行平均，对于轨迹文件处理有几个注意事项。

1. mdapy不能读取一个文件中包含多个Frame的格式，用户需要手动对轨迹文件进行切割，始终牢记mdapy是为了处理单frame设计的
2. mdapy会把轨迹中的所有frame都读取到内存中来，所以用户要谨慎思考自己的体系是否需要那么多frame
3. 为了计算MSD和Lindemann系数，mdapy会自动将轨迹中的坐标根据边界条件展开且会对原子序号排序，只有计算这两个参数，用户才应该使用MultiSystem,其余情况都应该使用System


- 这里我们读取rdf.0.dump到rdf.4.dump总共5个frames来进行计算

In [ ]:
MS = mp.MultiSystem([f'traj/rdf.{i}.dump' for i in range(5)])

In [ ]:
MS[0]

### 计算MSD

In [ ]:
MS.cal_mean_squared_displacement()

In [ ]:
fig, ax = MS.MSD.plot()

### 计算Lindemann index

In [ ]:
MS.cal_lindemann_parameter()

In [ ]:
fig, ax = MS.Lindemann.plot()

mdapy会把每个原子在MSD和lindemann系数上的贡献保存下来

In [ ]:
MS[0]

用户可以把结果保存到一系列dump文件中

In [ ]:
MS.write_dumps()

# 实战案例：计算层错能

这里我们以Al为例，计算他在<112>方向滑移的层错能。

## 建立一个晶向为x[11-2], y[1-10], z[111]的构型

In [ ]:
lattice_constant = 4.05
lattice_type = 'FCC'
fcc = mp.LatticeMaker(lattice_constant, 
                      lattice_type, 
                      10, 15, 10, 
                      crystalline_orientation=
                      np.array([[1, 1, -2], 
                                [1, -1, 0], 
                                [1, 1, 1]]))
fcc.compute()
system = mp.System(pos=fcc.pos, box=fcc.box)

In [ ]:
system.display()

## 将z方向改为自由边界

In [ ]:
system.boundary[-1] = 0

## 计算初始的势能

In [ ]:
system.cal_energy_force('frame/Al_DFT.eam.alloy', ['Al'], max_neigh=150)
pe0 = system.data['pe'].sum()

## 定义每一次移动的间隔距离

In [ ]:
move = (lattice_constant/6**0.5+0.2)/50
factor = system.box[0,0] * system.box[1,1]/16021.766200000002 # 转换系数

## 开始循环计算

In [ ]:
sfe = [0.]
for i in tqdm(range(1, 51)):
    newdata = system.data.with_columns(
        pl.when(pl.col('z')>35).
        then(pl.col('x')+move).
        otherwise(pl.col('x')).
        alias('x'))
    system.update_data(newdata, update_pos=True) # 移动上半部分
    system.wrap_pos() # 周期性折叠
    system.if_neigh = False # 确保重新建立邻域
    system.cal_energy_force('frame/Al_DFT.eam.alloy', ['Al'], max_neigh=150)
    sfe.append((system.data['pe'].sum()-pe0)/factor)

## 绘制结果

In [ ]:
fig, ax = mp.set_figure(figsize=(10, 7), use_pltset=True)

plt.plot(np.arange(51)*move/(lattice_constant/6**0.5), sfe, 'o-')
plt.xlabel("Displacement along $<112>$ " + "$\mathregular{(a_0/\sqrt{6})}$")
plt.ylabel("SFE ($\mathregular{mJ/{m^2}}$)")
plt.text(0.47, 100, 'USF')
plt.text(0.97, 25, 'ISF')

# 发展mdapy

mdapy目前还是一个在快速迭代发展的程序，但是由于我个人时间有限，难免会存在一些bug，比如文档错误，代码在有些情况会出错等。
大家可以通过在github主页上面提交issue进行反馈。我有时间的话会尽快修复。如果有一些功能在mdapy中没有实现，但是又确实很常用，
也可以在issue里面反馈。

> 最后，如果你喜欢mdapy，请给它Star。如果你在文章中使用了mdapy，请引用它。

<img src="image/activities.jpg" alt="activities.jpg" border="0" width="800"/>